In [1]:
import os
import time
import cv2
import requests
import json
import numpy as np
import tensorflow as tf
from models.data_prepare import load_and_preprocess_image_pridict
from models.config import TABLE_PATH, TEST_PATH

## 配置信息

In [2]:
#字典
with open(TABLE_PATH, 'r',encoding='gbk') as f:
    inv_table = [char.strip() for char in f]+[' ']*2

In [3]:
#测试数据集
test_all_image_paths = [TEST_PATH + img for img in sorted(os.listdir(TEST_PATH))]

## 调用tensorflow serving方法

In [4]:
def text_ocr(image_list):
    '''
    矩形框文字识别
    '''
    result_list = []
    input_image = []
    #输入的batch图片中的最大长度
    max_length = max([int(32*img.shape[1]/img.shape[0]) for img in image_list])
    #先保持长宽比，将图片高度resize到32，再在图片末尾进行0填充，保持同样图片长度
    for img in image_list:
        image = cv2.resize(img, (int(32*img.shape[1]/img.shape[0]), 32), 3)
        padding_im = np.zeros((32, max_length, 3), dtype=np.float32)
        padding_im[:, 0:img.shape[1],:] = image
        input_image.append(padding_im.tolist())
    url = 'http://192.168.46.230:7501/v1/models/crnn:predict'
    headers = {"content-type": "application/json"}
    data = json.dumps({
            "signature_name": 'serving_default',
            "instances":input_image})
    predictions = requests.post(url,data=data, headers=headers,timeout=50).json()
    if 'predictions' in predictions:
        if len(predictions['predictions']) > 0:
            for sentence in predictions['predictions']:
                result = ''
                for i in range(len(sentence)):
                    char_index = np.argmax(sentence[i])
                    if char_index != len(inv_table):
                        result +=  inv_table[char_index]
                result_list.append(result)
    return result_list

## 准备图片

In [5]:
image_list = []
for i in range(81,90):
    image_list.append(load_and_preprocess_image_pridict(test_all_image_paths[i], mode = 'client'))

## 调用接口

In [6]:
start = time.time()
result = text_ocr(image_list)
print('识别时间：', time.time()-start)
print('识别结果：', result)

识别时间： 27.314571142196655
识别结果： ['检查 期:2019-12-30', '检查医生:迟琼', '项目名称', '检查结果', '单位', '提示', '参考值', '1.32', 'PY.乙肝病毒表面抗原定量']
